# Week 10 — Efficient Training (Training at Scale)

This notebook focuses on building efficient training pipelines. You'll:
- Implement efficient DataLoaders with prefetching
- Profile and optimize data loading bottlenecks
- Use learning rate schedulers
- Build robust checkpoint systems

In [ ]:
# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import time
from pathlib import Path
%matplotlib inline

torch.manual_seed(42)
print(f"PyTorch version: {torch.__version__}")

In [ ]:

import os, pickle

CACHE_DIR = "cache_week10"
os.makedirs(CACHE_DIR, exist_ok=True)

def save_result(key, obj):
    with open(os.path.join(CACHE_DIR, f"{key}.pkl"), "wb") as f:
        pickle.dump(obj, f)

def load_result(key):
    path = os.path.join(CACHE_DIR, f"{key}.pkl")
    if os.path.exists(path):
        with open(path, "rb") as f:
            return pickle.load(f)
    return None

def cached(key, compute_fn):
    result = load_result(key)
    if result is not None:
        print(f"[cache] loaded '{key}'")
        return result
    print(f"[cache] computing '{key}'...")
    result = compute_fn()
    save_result(key, result)
    return result

print("Cache utilities ready. Results will be stored in:", CACHE_DIR)


## 1. Custom Dataset with Efficient Loading

Implement a custom dataset with proper data loading patterns.

In [ ]:
# Custom dataset with simulated I/O delay
class SyntheticDataset(Dataset):
    def __init__(self, n_samples=1000, n_features=100, delay_ms=0):
        """
        Args:
            n_samples: Number of samples
            n_features: Number of features
            delay_ms: Simulated I/O delay in milliseconds
        """
        self.n_samples = n_samples
        self.n_features = n_features
        self.delay_ms = delay_ms
        
        # Pre-generate data (in practice, you'd load from disk)
        self.data = torch.randn(n_samples, n_features)
        self.labels = torch.randint(0, 10, (n_samples,))
    
    def __len__(self):
        return self.n_samples
    
    def __getitem__(self, idx):
        # Simulate I/O delay
        if self.delay_ms > 0:
            time.sleep(self.delay_ms / 1000.0)
        
        return self.data[idx], self.labels[idx]

# Create datasets with different delays
dataset_fast = SyntheticDataset(n_samples=1000, delay_ms=0)
dataset_slow = SyntheticDataset(n_samples=1000, delay_ms=1)

print(f"Dataset size: {len(dataset_fast)}")
sample_x, sample_y = dataset_fast[0]
print(f"Sample shape: {sample_x.shape}, label: {sample_y}")

## 2. DataLoader Performance Comparison

Compare different DataLoader configurations and measure throughput.

In [ ]:

def benchmark_dataloader(dataset, batch_size=32, num_workers=0, pin_memory=False, n_batches=50):
    import time
    loader = DataLoader(dataset, batch_size=batch_size, num_workers=num_workers,
                        pin_memory=pin_memory, shuffle=True)
    start = time.time()
    for i, (batch_x, _) in enumerate(loader):
        if i >= n_batches: break
        _ = batch_x.mean()
    elapsed = time.time() - start
    return elapsed, n_batches / elapsed

configs = [
    {'num_workers': 0, 'pin_memory': False, 'name': '0 workers'},
    {'num_workers': 2, 'pin_memory': False, 'name': '2 workers'},
    {'num_workers': 4, 'pin_memory': False, 'name': '4 workers'},
    {'num_workers': 4, 'pin_memory': True,  'name': '4 workers + pin_memory'},
]

def _run_benchmarks():
    results = []
    for cfg in configs:
        elapsed, tput = benchmark_dataloader(dataset_slow, 32, cfg['num_workers'], cfg['pin_memory'])
        results.append({'name': cfg['name'], 'elapsed': elapsed, 'throughput': tput})
        print(f"{cfg['name']:28s}: {elapsed:.2f}s ({tput:.1f} batches/s)")
    return results

dataloader_results = cached("dataloader_benchmark_4configs", _run_benchmarks)

names      = [r['name']       for r in dataloader_results]
throughput = [r['throughput'] for r in dataloader_results]

plt.figure(figsize=(10, 5))
plt.barh(names, throughput, color='skyblue')
plt.xlabel('Throughput (batches/s)'); plt.title('DataLoader Performance Comparison')
plt.grid(axis='x', alpha=0.3); plt.tight_layout(); plt.show()
print("\n→ Using multiple workers significantly improves throughput!")


## 3. Learning Rate Schedulers

Experiment with different LR scheduling strategies.

In [ ]:
# Simple model for demonstration
model = nn.Sequential(
    nn.Linear(100, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

# Create different schedulers
def visualize_schedulers(initial_lr=0.1, n_epochs=100):
    schedulers_to_test = {
        'StepLR': optim.lr_scheduler.StepLR(
            optim.SGD(model.parameters(), lr=initial_lr),
            step_size=30,
            gamma=0.1
        ),
        'ExponentialLR': optim.lr_scheduler.ExponentialLR(
            optim.SGD(model.parameters(), lr=initial_lr),
            gamma=0.95
        ),
        'CosineAnnealingLR': optim.lr_scheduler.CosineAnnealingLR(
            optim.SGD(model.parameters(), lr=initial_lr),
            T_max=n_epochs
        ),
        'ReduceLROnPlateau': optim.lr_scheduler.ReduceLROnPlateau(
            optim.SGD(model.parameters(), lr=initial_lr),
            mode='min',
            factor=0.5,
            patience=10
        )
    }
    
    lr_histories = {name: [] for name in schedulers_to_test.keys()}
    
    for epoch in range(n_epochs):
        for name, scheduler in schedulers_to_test.items():
            current_lr = scheduler.optimizer.param_groups[0]['lr']
            lr_histories[name].append(current_lr)
            
            # Step scheduler
            if name == 'ReduceLROnPlateau':
                # Simulate a metric (e.g., loss)
                fake_loss = 1.0 / (epoch + 1) + np.random.randn() * 0.01
                scheduler.step(fake_loss)
            else:
                scheduler.step()
    
    return lr_histories

# Visualize
lr_histories = visualize_schedulers(initial_lr=0.1, n_epochs=100)

plt.figure(figsize=(12, 5))
for name, lrs in lr_histories.items():
    plt.plot(lrs, label=name, linewidth=2)

plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.title('Learning Rate Schedulers Comparison')
plt.legend()
plt.grid(alpha=0.3)
plt.yscale('log')
plt.tight_layout()
plt.show()

## 4. Robust Checkpointing System

Build a comprehensive checkpoint system with metadata and resumption support.

In [ ]:
# Checkpoint manager
class CheckpointManager:
    def __init__(self, checkpoint_dir='checkpoints', keep_last_n=3):
        self.checkpoint_dir = Path(checkpoint_dir)
        self.checkpoint_dir.mkdir(exist_ok=True)
        self.keep_last_n = keep_last_n
    
    def save(self, model, optimizer, scheduler, epoch, metrics, is_best=False):
        """
        Save checkpoint with metadata
        
        Args:
            model: PyTorch model
            optimizer: Optimizer state
            scheduler: LR scheduler state
            epoch: Current epoch
            metrics: Dict of metrics (e.g., {'train_loss': 0.5, 'val_acc': 0.9})
            is_best: Whether this is the best model so far
        """
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
            'metrics': metrics,
        }
        
        # Save regular checkpoint
        checkpoint_path = self.checkpoint_dir / f'checkpoint_epoch_{epoch}.pth'
        torch.save(checkpoint, checkpoint_path)
        print(f"Checkpoint saved: {checkpoint_path}")
        
        # Save best model
        if is_best:
            best_path = self.checkpoint_dir / 'best_model.pth'
            torch.save(checkpoint, best_path)
            print(f"Best model saved: {best_path}")
        
        # Clean up old checkpoints
        self._cleanup_old_checkpoints()
    
    def load(self, checkpoint_path, model, optimizer=None, scheduler=None):
        """
        Load checkpoint and restore state
        """
        checkpoint = torch.load(checkpoint_path)
        
        model.load_state_dict(checkpoint['model_state_dict'])
        if optimizer:
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        if scheduler and checkpoint['scheduler_state_dict']:
            scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        
        return checkpoint['epoch'], checkpoint['metrics']
    
    def _cleanup_old_checkpoints(self):
        """Keep only the last N checkpoints"""
        checkpoints = sorted(self.checkpoint_dir.glob('checkpoint_epoch_*.pth'))
        if len(checkpoints) > self.keep_last_n:
            for ckpt in checkpoints[:-self.keep_last_n]:
                ckpt.unlink()
                print(f"Removed old checkpoint: {ckpt}")

# Example usage
ckpt_manager = CheckpointManager(checkpoint_dir='./checkpoints', keep_last_n=3)

# Simulate training and saving checkpoints
model = nn.Linear(10, 2)
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10)

best_val_acc = 0.0
for epoch in range(1, 6):
    # Simulate training
    train_loss = 1.0 / epoch + np.random.rand() * 0.1
    val_acc = min(0.95, 0.5 + epoch * 0.1 + np.random.rand() * 0.05)
    
    metrics = {'train_loss': train_loss, 'val_acc': val_acc}
    is_best = val_acc > best_val_acc
    
    if is_best:
        best_val_acc = val_acc
    
    ckpt_manager.save(model, optimizer, scheduler, epoch, metrics, is_best=is_best)
    scheduler.step()
    print()

# Load best model
print("\nLoading best model...")
best_epoch, best_metrics = ckpt_manager.load('./checkpoints/best_model.pth', model, optimizer, scheduler)
print(f"Loaded best model from epoch {best_epoch}")
print(f"Metrics: {best_metrics}")

## Exercises for Further Practice

1. **Mixed Precision Training**: Use `torch.cuda.amp` for faster training
2. **Gradient Accumulation**: Implement gradient accumulation for larger effective batch sizes
3. **Profiling**: Use PyTorch profiler to identify bottlenecks
4. **Distributed Training**: Explore `torch.nn.DataParallel` or `DistributedDataParallel`
5. **Real Dataset**: Apply these techniques to a large dataset (e.g., ImageNet subset)

## Deliverables Checklist

- [ ] Custom dataset with efficient data loading
- [ ] DataLoader performance benchmarks
- [ ] Learning rate scheduler experiments
- [ ] Robust checkpoint system implementation
- [ ] Short report on throughput optimizations

## Recommended Resources

- PyTorch documentation on data loading and DataLoader
- PyTorch performance tuning guide
- Papers on distributed training and mixed precision

In [ ]:

# EXERCISE 1 — Gradient Accumulation
# Goal: implement gradient accumulation over K=4 mini-batches, then call optimizer.step().
# This simulates a batch size 4× larger without increasing memory.
# Compare training curves with true batch_size=128 vs accum(batch=32, K=4).
# Use cached("grad_accum_vs_large_batch_50epochs", ...) to store both histories.
# Expected insight: accumulation mimics large batches at the cost of throughput.


In [ ]:

# EXERCISE 2 — Warmup + Cosine Schedule from Scratch
# Goal: implement a custom LR scheduler function (not using PyTorch schedulers):
#   - Linear warmup for 10 epochs from lr=0 to lr=0.1
#   - Cosine decay from epoch 10 to 100 back to lr=0
# Plot the LR curve and use it in a training loop.
# Expected insight: warmup avoids early instability when LR starts too high.


In [ ]:

# EXERCISE 4 — Resume-from-Checkpoint Training
# Goal: train a SimpleMLP for 25 epochs and save a checkpoint with CheckpointManager.
# Simulate a "crash" by reinitialising the model. Load the checkpoint and resume
# training for another 25 epochs. Confirm that the full 50-epoch loss curve is smooth.
# Use cached("resume_training_50epochs", ...) for the combined history.
# Expected insight: robust checkpointing is essential for long training runs.
